In [ ]:
import os

from dotenv import load_dotenv

In [ ]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_ENVIRON = os.environ['PINECONE_ENVIRON']


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
import openai

openai_api_key=OPENAI_API_KEY
model="text-embedding-ada-002"
embed_model=OpenAIEmbeddings(model=model,openai_api_key=openai_api_key)

In [ ]:
import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRON,
)

index_name="gpteach"

print(pinecone.describe_index(name=index_name))

In [ ]:
#vectorstore_cls=Pinecone.from_existing_index(index_name=index_name,embedding=embed_model)

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('examples.csv')

# Convert DataFrame to dictionary
data_dict = df.to_dict(orient='records')
print(data_dict)


In [ ]:
column_names = df.columns.tolist()
print(column_names[1])


In [ ]:
# examples = data_dict

# example_prompt = PromptTemplate(
#     input_variables=[column_names[0], column_names[1]],
#     template=f"Input: {{{column_names[0]}}}\nOutput: {{{column_names[1]}}}",
# )
# print(example_prompt)

examples = data_dict

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
print(example_prompt)



In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # These are the examples it has available to choose from.
    examples=examples,
    # This is the PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings=embed_model,
    index_name=index_name,
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    vectorstore_cls=Pinecone,
    k=3,
)


In [ ]:
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give step by step repair instructions for every input",
    suffix="Input: {query}\nOutput: ", 
    input_variables=["query"],
) 

In [ ]:
# An example with small input, so it selects all examples.
print(dynamic_prompt.format(query="how do you repair a large puncture?"))

In [ ]:
# from langchain.chat_models import ChatOpenAI

# openai = ChatOpenAI(
#     model="gpt-3.5-turbo",
#     openai_api_key=OPENAI_API_KEY,
#     temperature=0.7
# )

In [ ]:
from langchain.llms import OpenAI

openai = OpenAI(
    model_name="text-davinci-003",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.7
)

In [ ]:
print(openai(dynamic_prompt.format(query="how do you repair an oversized hole?")))

In [ ]:
# You can add an example to an example selector as well.
new_example = {"input": "big", "output": "small"}
dynamic_prompt.example_selector.add_example(new_example)
print(dynamic_prompt.format(adjective="enthusiastic"))